In [46]:
from urllib import request
#from fake_useragent import UserAgent, VERSION
from urllib import parse
import json
import requests
import re
import pandas as pd
from openpyxl import Workbook
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm
ua = UserAgent(path='fake_useragent.json')

In [2]:
closedf = pd.read_feather('BasicFactor_Close.txt').set_index('time')
stockse = closedf.columns

In [79]:
#爬取雪球网上的公司信息
info_all = []
#每次只爬1000股 保存五个csv文件
stocklist = stockse[5000:]
for stock in tqdm(stocklist):
    stockcode = stock[-2:] + stock[:6]
    url = 'https://stock.xueqiu.com/v5/stock/f10/cn/company.json?symbol={}'.format(stockcode)
    uastr = ua.random
    headers = {'User-Agent':uastr,'Cookie':'xqat=197a3a870824d1754f6edf083d719bd1a3aabe88'}
    res = requests.get(url=url,headers=headers)
    info = json.loads(res.text)['data']['company']
    if info == None:
        info_all.append([])
        continue
    infolist = [info['org_name_cn'],info['org_short_name_cn'],info['pre_name_cn'],
                info['main_operation_business'],info['operating_scope'],info['org_cn_introduction'],
                info['legal_representative'],info['actual_controller'],info['office_address_cn']]
    info_all.append(infolist)
    time.sleep(1)
infodf = pd.DataFrame(info_all)
infodf.columns = ['name','shortname','prename','operation','operation_scope','introduction','legal_representative','actual_controller','office']
infodf.index = stocklist

100%|██████████| 432/432 [08:39<00:00,  1.20s/it]


In [80]:
infodf.to_csv('info5.csv')

In [85]:
df0 = pd.read_csv('info0.csv')
df1 = pd.read_csv('info1.csv')
df2 = pd.read_csv('info2.csv')
df3 = pd.read_csv('info3.csv')
df4 = pd.read_csv('info4.csv')
df5 = pd.read_csv('info5.csv')

In [87]:
finaldf = pd.concat([df0,df1,df2,df3,df4,df5])
finaldf.to_csv('infoall.csv')